# SnowMaL pipeline

In [ ]:
import os, glob
import matplotlib.pyplot as plt
import datetime
import rioxarray as rxr
import yaml
from TopoPyScale import topoclass as tc


## Define paths in directory and settings

In [ ]:
# Site name
site_name = 'MCS'
# Snow depth date
sd_date = '2024-03-15'
# Path to the snowmal code repository
snowmal_path = '/Users/raineyaberle/Research/PhD/SnowDEMs/snowmal'
# Path to reference DEM
dem_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/MCS/refdem/MCS_REFDEM_WGS84_5m.tif'
# Path to snow depth map
sd_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/MCS/lidar/20240315_MCS-snowdepth_RF_5m.tif'
# Output path
out_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SnowMaL/study-sites/MCS/'

## 1. Downscale ERA5 data to reference DEM using [TopoPyScale]()

In [ ]:
# -----Define output directory for downscaled ERA5 data
era_out_dir = os.path.join(out_dir, 'ERA5')
if not os.path.exists(era_out_dir):
    os.mkdir(era_out_dir)
    print('Made directory')

# -----Load and modify example config file
config_fn = os.path.join(snowmal_path, 'inputs', 'topopyscale_config_example.yml')
with open(config_fn) as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
# Current date
config['project']['date'] = datetime.datetime.now().date()
# Paths in directory
config['project']['directory'] = era_out_dir
config['climate']['era5']['path'] = era_out_dir
config['dem']['path'] = os.path.dirname(dem_fn)
config['dem']['file'] = os.path.basename(dem_fn)
config['outputs']['directory'] = era_out_dir
# Output CRS
config['dem']['epsg'] = rxr.open_rasterio(dem_fn).rio.crs.to_epsg()
# Start and end dates
end_date = sd_date
end_dt = datetime.date(int(end_date[0:4]), int(end_date[5:7]), int(end_date[8:]))
start_year = (end_dt.year if end_dt.month >= 10 else end_dt.year-1)
start_dt = datetime.date(start_year, 10, 1)
config['project']['start'] = start_dt
config['project']['end'] = end_dt
# Save adjusted config file
adj_config_fn = os.path.join(era_out_dir, 'topopyscale_config.yml')
with open(adj_config_fn, 'w') as out_file:
    yaml.dump(config, out_file)
print('Adjusted config file saved:', adj_config_fn)

# -----Run TopoPyScale 
# Load Configuration
mp = tc.Topoclass(adj_config_fn)
# Compute parameters of the DEM (slope, aspect, sky view factor)
mp.compute_dem_param()
mp.extract_topo_param()
# compute solar geometry and horizon angles
mp.compute_solar_geometry()
mp.compute_horizon()
# Perform the downscaling
mp.downscale_climate()
# Export output to desired format
mp.to_netcdf()

## 2. Construct training data

## 3. Develop machine learning models

## 4. Construct snow depth predictions time series